#### Import libraries

In [1]:
from osgeo import gdal
from pct2rgb import pct2rgb

#### Load in the .tif file

In [5]:
file_path = "/Users/tannermccoy/Development/Work/charts/mapserv/charts/sec/20231130/Anchorage.tif"
tmp_file = "/Users/tannermccoy/Development/Work/charts/mapserv/charts/sec/20231130/tmp.tif"
data = gdal.Open(file_path)


#### Get information about the .tif file

In [10]:
wkt = data.GetProjection()
sr = gdal.osr.SpatialReference()
sr.ImportFromWkt(wkt)
p4 = sr.ExportToProj4()
srout = gdal.osr.SpatialReference()
srout.ImportFromEPSG(4326)
reproj = gdal.osr.CoordinateTransformation(sr, srout)
invreproj = gdal.osr.CoordinateTransformation(srout, sr)

In [14]:
rotated = 0
width, height = data.RasterXSize, data.RasterYSize # int
gt = data.GetGeoTransform() # tuple
minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5]
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3]

if minx > maxx:
    swap = maxx
    maxx = minx
    minx = swap

if miny > maxy:
    swap = maxy
    maxy = miny
    miny = swap

if (gt[2] != 0) or (gt[4] != 0):
    rotated = 1

In [6]:
rastercount = data.RasterCount
pct2rgb(src_filename=file_path, dst_filename=tmp_file)


0...10...20...30...40...50...60...70...80...90...100 - done.


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x10f417f00> >

In [9]:
gdal.Warp(destNameOrDestDS=tmp_file, srcDSOrSrcDSTab=file_path, dstAlpha=True)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x12526f9c0> >

In [ ]:
def inv_gt(gt_in) -> list:
    det = gt_in[1] * gt_in[5] - gt_in[2] * gt_in[4]

    if abs(det) < 0.000000000000001:
        return None
    
    inv_det = 1.0 / det

    gt_out = []
    gt_out.append((gt_in[2] * gt_in[3] - gt_in[0] * gt_in[5]) * inv_det)
    gt_out.append(gt_in[5] * inv_det)
    gt_out.append(-gt_in[2] * inv_det)
    gt_out.append((-gt_in[1] * gt_in[3] - gt_in[0] * gt_in[4]) * inv_det)
    gt_out.append(-gt_in[4] * inv_det)
    gt_out.append(gt_in[1] * inv_det)

    return gt_out

#### Check for crops

In [ ]:
igt = inv_gt(gt)
nocrop = 0
refcount = 0